In [88]:
import fnmatch
import os
from collections import Counter
import json

In [89]:
dir_to_int = {
    "N": 1,
    "S": 2,
    "E": 3,
    "W": 4,
}

In [90]:
matches = []
# the directory contains a list of PeMS station meta data files
for root, dirnames, filenames in os.walk('/home/conway/temp/meta'):
    for filename in fnmatch.filter(filenames, '*.txt'):
        matches.append(os.path.join(root, filename))

In [91]:
# sanity check to ensure files have been found
matches[:5]

['/home/conway/temp/meta/2009/d8/d08_text_meta_2009_12_22.txt',
 '/home/conway/temp/meta/2009/d8/d08_text_meta_2008_10_07.txt',
 '/home/conway/temp/meta/2009/d8/d08_text_meta_2009_10_15.txt',
 '/home/conway/temp/meta/2009/d8/d08_text_meta_2009_01_27.txt',
 '/home/conway/temp/meta/2009/d8/d08_text_meta_2009_07_30.txt']

In [92]:
# Create map of station id -> Counter
# The counter stores the counts of the station tuple (direction, lat, lon, freeway, district, s_type, name)
# This is needed because stations are reported multiple times and some statitions report incorrect/different information
stations_map = {}
for m in matches:
    with open(m) as f:
        for l in f.readlines()[1:]:
            vals = l.split("\t")
            station = int(vals[0])
            freeway = int(vals[1])
            direction = dir_to_int[vals[2]]
            district = int(vals[3])
            s_type = vals[11]
            name = vals[13]
            try:
                lat = float(vals[8])
                lon = float(vals[9])
            except:
                lat = None
                lon = None
            t = (direction, lat, lon, freeway, district, s_type, name)
            if station not in stations_map:
                stations_map[station] = Counter()
            stations_map[station][t] += 1

In [93]:
stations_map[1100310].most_common(1)[0][0]

(1, 32.767779, -117.205941, 5, 11, 'FR', 'SEA WORLD DR')

In [94]:
# Use the naive approach and select the most-common station tuple to represent each station
station_directions = []
for k, c in stations_map.items():
    t = c.most_common(1)[0][0]
    s = {'station': k, 'direction': t[0], 'freeway': t[3], 'district': t[4], 'type': t[5], 'name': t[6]}
    if t[1] is not None and t[2] is not None:
        s['latitude'] = t[1]
        s['longitude'] = t[2]
    station_directions.append(s)

In [95]:
station_directions[0]

{'direction': 1,
 'district': 8,
 'freeway': 215,
 'latitude': 33.785651,
 'longitude': -117.218642,
 'name': 'RTE 74/4th SEP N/O',
 'station': 819200,
 'type': 'ML'}

In [96]:
# WriteDictToCSV('station_direction.csv', ['station','direction'],station_directions)

In [97]:
json_filename = "station_direction.json"

with open(json_filename, 'w') as fp:
    json.dump(station_directions, fp)

In [98]:
with open(json_filename, 'r') as fp:
    d = json.load(fp)

In [99]:
from osgeo import ogr

# Entire US urban shape file
# https://www.census.gov/geo/reference/ua/urban-rural-2010.html
# http://www2.census.gov/geo/tiger/TIGER2010/UA/2010/tl_2010_us_uac10.zip
shapeFile = r"/home/conway/Downloads/tl_2010_us_uac10.shp"

driver = ogr.GetDriverByName('ESRI Shapefile')

dataSource = driver.Open(shapeFile, 0) # 0 means read-only. 1 means writeable.

# Check to see if shapefile is found.
if dataSource is None:
    print 'Could not open %s' % (shapeFile)
else:
    print 'Opened %s' % (shapeFile)
    layer = dataSource.GetLayer()
    featureCount = layer.GetFeatureCount()
    print "Number of features in %s: %d" % (os.path.basename(shapeFile),featureCount)

# Print field names for reference
layerDefinition = layer.GetLayerDefn()
for i in range(layerDefinition.GetFieldCount()):
    print layerDefinition.GetFieldDefn(i).GetName()

Opened /home/conway/Downloads/tl_2010_us_uac10.shp
Number of features in tl_2010_us_uac10.shp: 3592
UACE10
GEOID10
NAME10
NAMELSAD10
LSAD10
MTFCC10
UATYP10
FUNCSTAT10
ALAND10
AWATER10
INTPTLAT10
INTPTLON10


In [100]:
# Define transformations into shapefile's CRS
geo_ref = layer.GetSpatialRef()
point_ref=ogr.osr.SpatialReference()
point_ref.ImportFromEPSG(4326)
ctran=ogr.osr.CoordinateTransformation(point_ref,geo_ref)

layer.ResetReading() #Read the layer from the beginning again
# for feature in layer:
#     geom = feature.GetGeometryRef()
#     print feature.GetField("NAME10")

def check(lon, lat):
    #Transform incoming longitude/latitude to the shapefile's projection
    [lon,lat,z]=ctran.TransformPoint(lon,lat,0.0)

    #Create a point
    pt = ogr.Geometry(ogr.wkbPoint)
    pt.SetPoint_2D(0, lon, lat)

    #Set up a spatial filter such that the only features we see when we
    #loop through "lyr_in" are those which overlap the point defined above
    layer.SetSpatialFilter(pt)

    # overlapped feature
    for feat_in in layer:
        # Assuming there's only one overlapping feature
        return feat_in
    
    return None

def getUrbanType(feat):
    # Field NAMELSAD10 contains the urbanized area name and type
    # For example, "San Diego, CA Urbanized Area" or "Brawley, CA Urban Cluster"
    type = feat.GetField("NAMELSAD10").split(" ")[-1] # "Area" or "Cluster"
    if type not in ("Area", "Cluster"):
        raise ValueError("Unknown urban type: " + type)
    else:
        if type == "Area":
            return 1
        else:
            return 2

In [101]:
# Sanity check to ensure point in San Diego County returns 1 (i.e. Urbanized Area)
feat = check(-117.01033,32.71483)
getUrbanType(feat)

1

In [102]:
i = 0
for station in d:
    try: 
        lat = station['latitude']
        lon = station['longitude']
        
        feat = check(lon, lat)
        
        if feat:
#             urban_enum = getUrbanType(feat) # 1 = urban area, 2 = urban cluster
            station['urban'] = 1
        else:
            station['urban'] = 0
            
    except Exception as error:
        # Not all stations have a reported lat/lon
        print('caught this error: ' + repr(error))
        continue
    i = i + 1
    if i % 1000 == 0:
        print i

caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
1000
caught this error: KeyError('latitude',)
2000
caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
3000
caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
4000
caught this error: KeyError('latitude',)
5000
6000
caught this error: KeyError('latitude',)
7000
8000
9000
caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
10000
11000
12000
13000
caught this error: KeyError('latitude',)
14000
15000
caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
caught this error

In [103]:
json_filename = "station_meta2.json"
with open(json_filename, 'w') as fp:
    json.dump(d, fp)

In [104]:
with open(json_filename, 'r') as fp:
    d = json.load(fp)

In [105]:
ml_stations = [v for v in d if v['type'] == 'ML']
dev_null = map(lambda x:  x.pop("type"), ml_stations)

In [106]:
len(ml_stations)

10633

In [107]:
ml_stations[0]

{u'direction': 1,
 u'district': 8,
 u'freeway': 215,
 u'latitude': 33.785651,
 u'longitude': -117.218642,
 u'name': u'RTE 74/4th SEP N/O',
 u'station': 819200,
 u'urban': 1}

In [108]:
json_filename = "station_meta_ML.json"
with open(json_filename, 'w') as fp:
    json.dump(ml_stations, fp)

In [109]:
with open(json_filename, 'r') as fp:
    d = json.load(fp)
    
import os
from osgeo import ogr

shapeFile = r"/home/conway/Downloads/tl_2010_06_zcta510.shp"

driver = ogr.GetDriverByName('ESRI Shapefile')

dataSource = driver.Open(shapeFile, 0) # 0 means read-only. 1 means writeable.

# Check to see if shapefile is found.
if dataSource is None:
    print 'Could not open %s' % (shapeFile)
else:
    print 'Opened %s' % (shapeFile)
    layer = dataSource.GetLayer()
    featureCount = layer.GetFeatureCount()
    print "Number of features in %s: %d" % (os.path.basename(shapeFile),featureCount)
    
layerDefinition = layer.GetLayerDefn()

for i in range(layerDefinition.GetFieldCount()):
    print layerDefinition.GetFieldDefn(i).GetName()
    
geo_ref = layer.GetSpatialRef()
point_ref=ogr.osr.SpatialReference()
point_ref.ImportFromEPSG(4326)
ctran=ogr.osr.CoordinateTransformation(point_ref,geo_ref)

layer.ResetReading() #Read the layer from the beginning again

# Define a function that given a latitude & longitude, returns is associated zip code
# Lookup is via a U.S. Census TIGER/Line shapefile
def getZip(lon, lat):
    #Transform incoming longitude/latitude to the shapefile's projection
    [lon,lat,z]=ctran.TransformPoint(lon,lat,0.0)

    #Create a point
    pt = ogr.Geometry(ogr.wkbPoint)
    pt.SetPoint_2D(0, lon, lat)

    #Set up a spatial filter such that the only features we see when we
    #loop through "lyr_in" are those which overlap the point defined above
    layer.SetSpatialFilter(pt)

    # overlapped feature
    for feat_in in layer:
        # Assuming there's only one overlapping feature
        #return feat_in
        return feat_in.GetField("ZCTA5CE10")
    
    return None

Opened /home/conway/Downloads/tl_2010_06_zcta510.shp
Number of features in tl_2010_06_zcta510.shp: 1769
STATEFP10
ZCTA5CE10
GEOID10
CLASSFP10
MTFCC10
FUNCSTAT10
ALAND10
AWATER10
INTPTLAT10
INTPTLON10
PARTFLG10


In [110]:
zip = getZip(-117.104,32.82)
print zip

92124


In [111]:
i = 0
for station in d:
    try: 
        lat = station['latitude']
        lon = station['longitude']
        
        zipcode = getZip(lon, lat)
        
        if zipcode:
            station['zip'] = zipcode
            
    except Exception as error:
        # Not all stations have a reported lat/lon
        print('caught this error: ' + repr(error))
        continue
    i = i + 1
    if i % 1000 == 0:
        print i

caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
1000
caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)
2000
caught this error: KeyError('latitude',)
3000
4000
caught this error: KeyError('latitude',)
5000
caught this error: KeyError('latitude',)
6000
7000
caught this error: KeyError('latitude',)
8000
caught this error: KeyError('latitude',)
9000
10000
caught this error: KeyError('latitude',)
caught this error: KeyError('latitude',)


In [112]:
json_filename = "station_meta_ML2.json"
with open(json_filename, 'w') as fp:
    json.dump(d, fp)

In [113]:
import pandas as pd

In [114]:
station_df = pd.read_json(json_filename)

In [115]:
station_df.head()

,direction,district,freeway,latitude,longitude,name,station,urban,zip
0,1,8,215,33.785651,-117.218642,RTE 74/4th SEP N/O,819200,1,92570
1,2,8,215,33.785554,-117.218747,RTE 74/4th SEP N/O,819201,1,92570
2,3,11,52,32.841682,-117.121715,W/O I-15,1114114,1,92122
3,1,8,215,33.782061,-117.213169,RTE 74/4th SEP S/O,819203,0,92570
4,1,8,215,33.889640,-117.270822,VAN BUREN BLVD,819204,0,92518


In [116]:
zipcode_density_df = pd.read_csv('/home/conway/temp/Zipcode-ZCTA-Population-Density-And-Area-Unsorted.csv')

In [117]:
zipcode_density_df.head()

,Zip/ZCTA,2010 Population,Land-Sq-Mi,Density Per Sq Mile
0,601,0,64.348,0
1,602,0,30.613,0
2,603,0,31.616,0
3,606,0,42.309,0
4,610,0,35.916,0


In [118]:
merged_df = pd.merge(station_df, zipcode_density_df[['Zip/ZCTA','Density Per Sq Mile']], how='left', left_on='zip', right_on='Zip/ZCTA')
merged_df.drop(['Zip/ZCTA'], axis=1, inplace=True)

In [119]:
merged_df.tail()

,direction,district,freeway,latitude,longitude,name,station,urban,zip,Density Per Sq Mile
10628,1,8,215,34.117224,-117.302924,10th STREET,819195,1,92411,6412.426614
10629,3,11,52,32.845413,-117.209402,.6 M E/O REGENTS RD,1114108,1,92117,5841.147019
10630,1,8,215,34.149161,-117.321884,27th STREET,819197,1,92405,6109.394837
10631,1,12,261,33.766351,-117.758414,HANDY 1,1212414,1,92602,2190.079479
10632,2,12,261,33.766516,-117.758825,HANDY 1,1212415,1,92602,2190.079479


In [120]:
merged_df.to_json('station_meta_density.json')